In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
import geopandas as gpd
import textwrap
from geojson import Feature, FeatureCollection, Point

In [2]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [3]:
df = pd.read_csv('Miami Neighborhoods New Construction - Edgewater.csv')

In [4]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('Edgewater.kml', driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\array.py:524: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return _unary_geo("centroid", self)
C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\geopandas\geoseries.py:133: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


In [5]:
project_gdf

,Name,Description,geometry,center_point
0,Edgewater,,"POLYGON Z ((-80.18541 25.79247 0.00000, -80.18...",POINT (-80.18934 25.80105)


In [6]:
df

,address,developer,units,acerage,price_sale,price_loan,loan_financer,category,description,story_link
0,2121 North Bayshore Drive,Two Roads Development,NaN,3.5,150000000.0,150000000,Lionheart Strategic Management LLC (45M) and F...,proposed,"<a href=""https://therealdeal.com/miami/2022/05...",https://therealdeal.com/miami/2022/05/24/two-r...
1,423 Northeast 27th Street,Groupe City,na,na,14800000.0,na,na,proposed,"<a href=""https://therealdeal.com/miami/2022/05...",https://therealdeal.com/miami/2022/05/05/vive-...
2,430 Northeast 31st Street,"Grupo Diana, Korner",279,NaN,NaN,55000000,Madison Realty Capital,under construction,"<a href=""https://therealdeal.com/miami/2022/04...",https://therealdeal.com/miami/2022/04/21/colom...
3,2900 Biscayne Boulevard in Miami,Crescent Heights,588 units,NaN,NaN,224000000,Blackstone,under construction,"<a href=""https://therealdeal.com/miami/2022/04...",https://therealdeal.com/miami/2022/04/01/russe...
4,2201 Northeast Second Avenue,Trilogy Real Estate Group,na,1.6,3060000.0,na,na,planned,"<a href=""https://therealdeal.com/miami/2022/03...",https://therealdeal.com/miami/2022/03/22/chica...
5,3333 Biscayne Boulevard,"Beitel Group, Aimco",600,2.8,46700000.0,na,na,proposed,"<a href=""https://therealdeal.com/miami/2022/03...",https://therealdeal.com/miami/2022/03/14/aimco...
6,2140 Northeast Second Avenue,Heritage Equity Partners,120,na,6300000.0,na,na,proposed,"<a href=""https://therealdeal.com/miami/2022/02...",https://therealdeal.com/miami/2022/02/14/herit...
7,710 Northeast 29th Street,"Michael Konig, Alex Posth, Kevin Venger, Louis...",70,na,22000000.0,na,na,proposed,"<a href=""https://therealdeal.com/miami/2022/01...",https://therealdeal.com/miami/2022/01/28/one-t...
8,2000 Biscayne Boulevard,"Kushner, PTM Partners",1300,na,58100000.0,127000000,Valley National Bank and AIG Asset Management,under construction,"<a href=""https://therealdeal.com/miami/2022/01...",https://therealdeal.com/miami/2022/01/18/kushn...
9,1900 Biscayne Boulevard,"Kushner, PTM Partners",1300,na,58100000.0,127000000,Valley National Bank and AIG Asset Management,under construction,"<a href=""https://therealdeal.com/miami/2022/01...",https://therealdeal.com/miami/2022/01/18/kushn...


In [7]:
df['geo_address'] = df['address'] + ', Miami FL'

In [8]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [9]:
df['loc']

0     (2121 N Bayshore Dr, Miami, FL 33137, USA, (25...
1     (423 NE 27th St, Miami, FL 33137, USA, (25.803...
2     (430 NE 31st St, Miami, FL 33137, USA, (25.806...
3     (2900 Biscayne Blvd, Miami, FL 33137, USA, (25...
4     (2201 NE 2nd Ave, Miami, FL 33137, USA, (25.79...
5     (3333 Biscayne Blvd, Miami, FL 33137, USA, (25...
6     (2140 NE 2nd Ave, Miami, FL 33137, USA, (25.79...
7     (710 NE 29th St, Miami, FL 33137, USA, (25.804...
8     (2000 Biscayne Blvd, Miami, FL 33137, USA, (25...
9     (1900 Biscayne Blvd, Miami, FL 33132, USA, (25...
10    (1836 Biscayne Blvd, Miami, FL 33132, USA, (25...
11    (2634 NE 2nd Ave, Miami, FL 33137, USA, (25.80...
12    (192 NE 27th St, Miami, FL 33137, USA, (25.802...
13    (186 NE 27th St, Miami, FL 33137, USA, (25.802...
14    (480 NE 29th St, Miami, FL 33137, USA, (25.804...
15    (456 NE 29th St, Miami, FL 33137, USA, (25.804...
16    (496 NE 26th St, Miami, FL 33137, USA, (25.801...
17    (116 NE 24th St, Miami, FL 33137, USA, (25

In [10]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [12]:
df['image_img'] = ' '

In [13]:
df.at[0,'image_img']=('<img src="TwoRoadsDevelopment.jpg" width="256" height="156">')
df.at[1,'image_img']=('<img src="GroupeCity.jpg" width="256" height="156">')
df.at[2,'image_img']=('<img src="GrupoDianaKorner.jpg" width="256" height="156">')
df.at[3,'image_img']=('<img src="CrescentHeights.jpg" width="256" height="156">')
df.at[4,'image_img']=('<img src="TrilogyRealEstate.jpg" width="256" height="156">')
df.at[5,'image_img']=('<img src="BeitelGroupAimco.jpg" width="256" height="156">')
df.at[6,'image_img']=('<img src="HeritageEquityPartners.jpg" width="256" height="156">')
df.at[7,'image_img']=('<img src="MichaelKonigAndCrew.jpg" width="256" height="156">')
df.at[8,'image_img']=('<img src="KushnerPTMpartners.jpg" width="256" height="156">')
df.at[9,'image_img']=('<img src="KushnerPTMpartners.jpg" width="256" height="156">')
df.at[10,'image_img']=('<img src="SkylightRealEstate.jpg" width="256" height="156">')
df.at[11,'image_img']=('<img src="TrilogyRealEstatex3.jpg" width="256" height="156">')
df.at[12,'image_img']=('<img src="TrilogyRealEstatex3.jpg" width="256" height="156">')
df.at[13,'image_img']=('<img src="TrilogyRealEstatex3.jpg" width="256" height="156">')
df.at[14,'image_img']=('<img src="SBdevelopmentGroup.jpg" width="256" height="156">')
df.at[15,'image_img']=('<img src="SBdevelopmentGroup.jpg" width="256" height="156">')
df.at[16,'image_img']=('<img src="SBdevelopmentGroup.jpg" width="256" height="156">')
df.at[17,'image_img']=('<img src="OmniNewYork.png" width="256" height="156">')
df.at[18,'image_img']=('<img src="MeloGroup.jpg" width="256" height="156">')
df.at[19,'image_img']=('<img src="ArchCompanies.jpg" width="256" height="156">')
df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')

In [14]:
df.columns

Index(['address', 'developer', 'units', 'acerage', 'price_sale', 'price_loan',
       'loan_financer', 'category', 'description', 'story_link', 'geo_address',
       'loc', 'point', 'lat', 'lon', 'altitude', 'image_img'],
      dtype='object')

How to customize HTML in folium popups:
https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a

In [15]:
def popup_html(row):
    i = row
    address = df['address'].iloc[i]
    developer = df['developer'].iloc[i]
    description = df['description'].iloc[i]
    image = df['image_img'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    {}'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


In [16]:

# def popup_html(row):
    
#     html = """<!DOCTYPE html>
# <html>
# <head>
# <h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(institution_name) + """
# </head>
#     <table style="height: 126px; width: 350px;">
# <tbody>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Institution Type</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(institution_type) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Institution URL</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(institution_url) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">City and State</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(city_state) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Highest Degree Awarded</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(highest_degree) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Admission Rate</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(admission_rate) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Annual Cost of Attendance $</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(cost) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">In-state Tuition $</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(instate_tuit) + """
# </tr>
# <tr>
# <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Out-of-state Tuition $</span></td>
# <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(outstate_tuit) + """
# </tr>
# </tbody>
# </table>
# </html>
# """
#     return html
# view rawDefine popup_html function.py hosted with ❤ by GitHub

In [17]:
df.lon

0    -80.186417
1    -80.187622
2    -80.187991
3    -80.189598
4    -80.190881
5    -80.189116
6    -80.190991
7    -80.185545
8    -80.189664
9    -80.189283
10   -80.189555
11   -80.191190
12   -80.191397
13   -80.191538
14   -80.186831
15   -80.187120
16   -80.187408
17   -80.192474
18   -80.185596
19   -80.188945
20   -80.186417
Name: lon, dtype: float64

In [18]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),min_zoom=14.5, zoom_start=15, min_lat=25.83,max_lat=25.79,
               scrollWheelZoom=True,
              min_lon=-80.21,max_lon=-80.17, max_bounds=True)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Edgewater')

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [19]:
m.save('index.html')